## Importing libraries

In [ ]:
import tradingeconomics as te
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
# import folium #no permission
import plotly.express as px
import plotly.graph_objects as go
import pycountry
import warnings
warnings.filterwarnings("ignore")

## Logging in into the API

In [ ]:
te.login(); # Signup for a free API key at https://developer.tradingeconomics.com/

# Setting the dataframe

In [ ]:
#list of all countries
countries = ['Afghanistan', 'Åland Islands', 'Albania', 'Algeria', 'American Samoa', 'Andorra', 'Angola', 'Anguilla', 'Antarctica', 'Antigua & Barbuda', 'Argentina', 'Armenia', 'Aruba', 'Australia', 'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bermuda', 'Bhutan', 'Bolivia', 'Bosnia & Herzegovina', 'Botswana', 'Bouvet Island', 'Brazil', 'British Indian Ocean Territory', 'British Virgin Islands', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean Netherlands', 'Cayman Islands', 'Central African Republic', 'Chad', 'Chile', 'China', 'Christmas Island', 'Cocos (Keeling) Islands', 'Colombia', 'Comoros', 'Congo - Brazzaville', 'Congo - Kinshasa', 'Cook Islands', 'Costa Rica', 'Côte d’Ivoire', 'Croatia', 'Cuba', 'Curaçao', 'Cyprus', 'Czechia', 'Denmark', 'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt', 'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia', 'Eswatini', 'Ethiopia', 'Falkland Islands', 'Faroe Islands', 'Fiji', 'Finland', 'France', 'French Guiana', 'French Polynesia', 'French Southern Territories', 'Gabon', 'Gambia', 'Georgia', 'Germany', 'Ghana', 'Gibraltar', 'Greece', 'Greenland', 'Grenada', 'Guadeloupe', 'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guinea-Bissau', 'Guyana', 'Haiti', 'Heard & McDonald Islands', 'Honduras', 'Hong Kong SAR China', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Isle of Man', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jersey', 'Jordan', 'Kazakhstan', 'Kenya', 'Kiribati', 'Kuwait', 'Kyrgyzstan', 'Laos', 'Latvia', 'Lebanon', 'Lesotho', 'Liberia', 'Libya', 'Liechtenstein', 'Lithuania', 'Luxembourg', 'Macao SAR China', 'Madagascar', 'Malawi', 'Malaysia', 'Maldives', 'Mali', 'Malta', 'Marshall Islands', 'Martinique', 'Mauritania', 'Mauritius', 'Mayotte', 'Mexico', 'Micronesia', 'Moldova', 'Monaco', 'Mongolia', 'Montenegro', 'Montserrat', 'Morocco', 'Mozambique', 'Myanmar (Burma)', 'Namibia', 'Nauru', 'Nepal', 'Netherlands', 'New Caledonia', 'New Zealand', 'Nicaragua', 'Niger', 'Nigeria', 'Niue', 'Norfolk Island', 'North Korea', 'North Macedonia', 'Northern Mariana Islands', 'Norway', 'Oman', 'Pakistan', 'Palau', 'Palestinian Territories', 'Panama', 'Papua New Guinea', 'Paraguay', 'Peru', 'Philippines', 'Pitcairn Islands', 'Poland', 'Portugal', 'Puerto Rico', 'Qatar', 'Réunion', 'Romania', 'Russia', 'Rwanda', 'Samoa', 'San Marino', 'São Tomé & Príncipe', 'Saudi Arabia', 'Senegal', 'Serbia', 'Seychelles', 'Sierra Leone', 'Singapore', 'Sint Maarten', 'Slovakia', 'Slovenia', 'Solomon Islands', 'Somalia', 'South Africa', 'South Georgia & South Sandwich Islands', 'South Korea', 'South Sudan', 'Spain', 'Sri Lanka', 'St. Barthélemy', 'St. Helena', 'St. Kitts & Nevis', 'St. Lucia', 'St. Martin', 'St. Pierre & Miquelon', 'St. Vincent & Grenadines', 'Sudan', 'Suriname', 'Svalbard & Jan Mayen', 'Sweden', 'Switzerland', 'Syria', 'Taiwan', 'Tajikistan', 'Tanzania', 'Thailand', 'Timor-Leste', 'Togo', 'Tokelau', 'Tonga', 'Trinidad & Tobago', 'Tunisia', 'Turkey', 'Turkmenistan', 'Turks & Caicos Islands', 'Tuvalu', 'U.S. Outlying Islands', 'U.S. Virgin Islands', 'Uganda', 'Ukraine', 'United Arab Emirates', 'United Kingdom', 'United States', 'Uruguay', 'Uzbekistan', 'Vanuatu', 'Vatican City', 'Venezuela', 'Vietnam', 'Wallis & Futuna', 'Western Sahara', 'Yemen', 'Zambia', 'Zimbabwe']

In [ ]:
#creating initial dataframe with first country
terr_df = te.getHistoricalData(country=countries[0],  indicator='Terrorism Index', output_type='df')

index = terr_df.index
#merging rest of countries
for country in countries[1:]:
    df = pd.DataFrame()
    df = df.append(te.getHistoricalData(country=country,  indicator='Terrorism Index'))
    if df.any().sum(): #if info exists for that country
        terr_df = pd.merge(terr_df, df, left_index = True, right_index = True, how = "outer")
#changing shape of dataframe    
terr_df = terr_df.stack(dropna = False).reset_index()

terr_df.columns = ["Year", "Country", "Terrorism Index"]
terr_df['Year'] = pd.to_datetime(terr_df['Year'])
terr_df.Year = terr_df['Year'].dt.year


Are there any null values?

In [ ]:
terr_df.loc[terr_df["Terrorism Index"][terr_df["Terrorism Index"].isna()].index]

Makes sense since south sudan was founded in 2011

In [ ]:
#creating dictionary with countries and their country iso codes 3 to use in the map
country_codes = {}
for country in pycountry.countries:
    for x in terr_df.Country.unique():
        try:
            if x == country.common_name:
                country_codes[x] = country.alpha_3
        except AttributeError:
            if x == country.name:
                country_codes[x] = country.alpha_3
#             elif x in country.name:
#                 country_codes[x] = country.alpha_3

In [ ]:
#Adding iso codes to the dataframe
terr_df["iso_alpha"] = 0
failed_countries = []
for i,country in enumerate(terr_df.Country):
    try:
        terr_df.iso_alpha[i] = country_codes[country]
    except KeyError:
        failed_countries.append(country)

In [ ]:
set(failed_countries)

In [ ]:
# we will enter codes for LAOS and South Korea manually since they  have different names registered
failed_dict = {"Laos":"LAO","South Korea": "KOR"}
# we will get the rest using a different approach (checking if country name in dataframe is in FULL NAME of the country in pycountries)
for country in pycountry.countries:
    for x in failed_countries:
        if x in country.name:
            failed_dict[x] = country.alpha_3
#adding the newly extracted iso codes to the dataframe
for x in failed_dict:
    for i in terr_df[terr_df.Country == x].index:
        terr_df.iso_alpha[i] = failed_dict[x]


## INTERACTIVE PLOTS
# Get time series plot of terrorism index across the years
#### run the next cell to input the countries of your choosing

In [ ]:
#input whether you want global data for or specific country
world = input("Do you want the average terrorism index for all countries or specific countries?\nPlease enter 'All' or 'Specific':").lower().strip()
while world not in ["all", "specific"]:
    world = input("Please enter 'All' or 'Specific':").lower().strip()
    
# producing plot with global terrorism index
if world == "all":
    terr = terr_df.groupby(["Year"])["Terrorism Index"].mean().reset_index()
    fig = px.line(terr, x = "Year", y = "Terrorism Index")
    fig.update_layout(height = 600, title="GLOBAL TERRORISM INDEX ACROSS THE YEARS\n", xaxis_title="YEAR", yaxis_title="TERRORISM INDEX",
                     template = "plotly_dark", title_font = dict(family="Rockwell", size=30))
    
# producing plot for specific input countries
elif world == "specific":
    country_plots = []
    number = input("For how many countries do you want to plot the Terrorism Index?\n Please enter a digit:").strip()
    while not number.isdigit():
        number = input("Please enter a digit:").strip()
    if int(number):
        for i in range(1, int(number) + 1):
            country = input(f"Country {i}:").title().strip()
            while country not in terr_df.Country.values:
                country = input(f"Country not in database\nCountry {i}:").title().strip()
            if not terr_df[terr_df.Country == country]["Terrorism Index"].any():
                print (f"Unfortunately there is no terrorism index data available for {country}")
            else:
                country_plots.append(country)
        terr = terr_df[terr_df.Country.isin(country_plots)]
        fig = px.line(terr, x = "Year", y = "Terrorism Index", color = "Country")

        fig.update_layout(height = 600, title="TERRORISM INDEX ACROSS THE YEARS\n", title_font = dict(family="Rockwell", size=30),
                          xaxis_title="YEAR", yaxis_title="TERRORISM INDEX",
                          legend_title="COUNTRY", template = "plotly_dark")

fig.show()

### We can see here with most countries, that there has been a trending increase in Terrorism Index after 2011 (except Russia) with Egypt and South Africa showing the most drastic increase.

# Get map plot of terrorsim index across countries (and specific year from 2006 to 2019) and show top 10 countries
#### run the next cell to input the year of your choosing

In [ ]:
#taking input of year/years of the data

year = input("Do you want terrorism index for a certain year or average of all years from 2006 to 2019?\n please enter a year number or 'ALL':").lower().strip()

#setting acceptable options 
options = [str(i) for i in range(2006, 2020)]
options.append("all")
while year not in options:
    year = (input("please enter a year from 2006 to 2019 or 'ALL'")).lower().strip()

# plot properties if all-time data is to be shown    
if year == "all":
    df = terr_df.groupby(["Country", "iso_alpha"])["Terrorism Index"].mean().reset_index()
    title = "Average Terrorism Index from 2006 to 2019  (gray = no data)"
    title2 = "Top 10 countries with highest average terrorism index from 2006 to 2019"
    
#plot properties for plots of a specific year
#title2 is title of second bar plot
else:    
    year = int(year)
    df = terr_df[terr_df.Year == year]
    title = f"Terrorism Index in {year}  (gray = no data)".upper()
    title2 = f"Top 10 countries with highest terrorism index in {year}"

#setting first plot of world map
fig = px.choropleth(df, locations=df.iso_alpha,
                    color="Terrorism Index",
                    hover_name="Country",
                    color_continuous_scale=px.colors.sequential.Reds, title = title,
                    width=1000, height=800)
fig.update_layout(coloraxis_colorbar=dict(len = 0.6),
                  geo = dict(landcolor = "gray"), template = "plotly_dark", title_font = dict(family="Rockwell", size=30))
fig.show()

#setting second plot showing bars of top 10 countries
df2 = df.sort_values("Terrorism Index", ascending = False ).dropna()[:10]
plt.figure(figsize = (20,8))
sb.set(rc={'axes.facecolor':'black'})
fig2 = sb.barplot(df2["Terrorism Index"], df2.Country, palette = "Reds_r", orient = "h")
fig2.grid(False);
plt.title(title2, fontsize = 30);

### We can see in the bar plot the countries with highest terrorism index specially Iraq and South Asian countries.

# The End

In [ ]:
# Author: Raghdaa